In [1]:
'''
Machine Learning para Analise de Dados
CESAR Scholl
Projeto de analise de credito
Recife, 2019


Equipe:
    Claudio Alves Monteiro
    Marcos Antonio Almeida Souto Júnior
    Virgínia Heimann
    Kayo Renato da Silva Nascimento
    Rosely Cabral
'''

'\nMachine Learning para Analise de Dados\nCESAR Scholl\nProjeto de analise de credito\nRecife, 2019\n\n\nEquipe:\n    Claudio Alves Monteiro\n    Marcos Antonio Almeida Souto Júnior\n    Virgínia Heimann\n    Kayo Renato da Silva Nascimento\n    Rosely Cabral\n'

In [2]:
# Spark Session
from pyspark.sql import SparkSession, Row
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [3]:
# import modules
import os
import pandas as pd
from pyspark.sql import functions as SF
import pyspark.sql.types as ST

# paths to spark and python3
os.environ['PYSPARK_SUBMIT_ARGS'] = '--executor-memory 1G pyspark-shell'
os.environ["SPARK_HOME"] = "/home/pacha/spark"
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"

In [4]:
# import data

df = spark.read.csv('data/jur.csv',
                    sep='\t',
                    encoding='utf-8',
                    header=True,
                    inferSchema=False)


df_cursor = spark.read.csv('data/Base_Des-TRN',
                    sep='\t',
                    encoding='utf-8',
                    header=True,
                    inferSchema=False)


In [5]:
df.printSchema()

root
 |-- CNPJ: string (nullable = true)
 |-- NOME_EMPRESA: string (nullable = true)
 |-- FANTASIA: string (nullable = true)
 |-- NATUREZA_JURIDICA: string (nullable = true)
 |-- ATIVIDADE_PRINCIPAL: string (nullable = true)
 |-- ATIVIDADES_SECUNDARIAS: string (nullable = true)
 |-- CNPJ_FORMATADO: string (nullable = true)
 |-- TIPO: string (nullable = true)
 |-- LOGRADOURO: string (nullable = true)
 |-- NUMERO: string (nullable = true)
 |-- COMPLEMENTO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- TELEFONE_1: string (nullable = true)
 |-- TELEFONE_2: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- ABERTURA: string (nullable = true)
 |-- CAPITAL_SOCIAL: string (nullable = true)
 |-- MOTIVO_SITUACAO: string (nullable = true)
 |-- SITUACAO: string (nullable = true)
 |-- DATA_SITUACAO: string (nullable = true)
 |-- SITUACAO_ESPECIAL: strin

In [6]:
df.show(10, False)

+--------------+-----------------------------------------------+---------------------+-----------------------------+----------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+------+------------------------+------+------------------------+------------------+--------+--------------+---+--------------+--------------+------------------------------------+------------------------+--------------+---------------+--------+------------------------+-----------------+----------------------+----+--------------------------------+-------------------------------------------------+------------------+---------------+----------+------------------------+------+----------+-------------+------------+-

In [7]:
df_cursor.show(5)

+--------------+--------+-------+
|          CNPJ|DATA_REF|CS_ALVO|
+--------------+--------+-------+
|11507110000160| 2016-10|      0|
|15062189000114| 2016-07|      0|
| 5841171000167| 2016-08|      0|
| 8792807000116| 2016-08|      0|
|24060073000137| 2017-01|      1|
+--------------+--------+-------+
only showing top 5 rows



In [6]:
#------ tratar inicio do CNPJ sem o 0

@SF.udf('string')
def fill_cnpj(value):
    aux = '00000000000000'+value
    return aux[len(value):]

#------ tratar inicio do CEP sem o 0

@SF.udf('string')
def fill_cep(value):
    aux = '00000000'+value
    return aux[len(value):]



In [7]:
df_cursor = df_cursor.withColumn('CNPJ',fill_cnpj('CNPJ'))
df_cursor.show(5)

+--------------+--------+-------+
|          CNPJ|DATA_REF|CS_ALVO|
+--------------+--------+-------+
|11507110000160| 2016-10|      0|
|15062189000114| 2016-07|      0|
|05841171000167| 2016-08|      0|
|08792807000116| 2016-08|      0|
|24060073000137| 2017-01|      1|
+--------------+--------+-------+
only showing top 5 rows



In [8]:
df = df.withColumn('CEP',fill_cep('CEP'))
df.show(5)

+--------------+--------------------+--------------------+--------------------+--------------------+----------------------+------------------+------+----------------+------+-----------+----------------+--------+--------------+---+--------------+--------------+--------------------+--------------------+--------------+---------------+--------+--------------------+-----------------+----------------------+----+--------------------+-----+--------------+---------------+----------+--------------------+------+----------+-------------+------------+--------------------+----+-----+
|          CNPJ|        NOME_EMPRESA|            FANTASIA|   NATUREZA_JURIDICA| ATIVIDADE_PRINCIPAL|ATIVIDADES_SECUNDARIAS|    CNPJ_FORMATADO|  TIPO|      LOGRADOURO|NUMERO|COMPLEMENTO|          BAIRRO|     CEP|     MUNICIPIO| UF|    TELEFONE_1|    TELEFONE_2|               EMAIL|            ABERTURA|CAPITAL_SOCIAL|MOTIVO_SITUACAO|SITUACAO|       DATA_SITUACAO|SITUACAO_ESPECIAL|DATA_SITUACAO_ESPECIAL| EFR|          NOME

In [9]:
df = df.withColumn('CNPJ',fill_cnpj('CNPJ'))
df.show(5)

+--------------+--------------------+--------------------+--------------------+--------------------+----------------------+------------------+------+----------------+------+-----------+----------------+--------+--------------+---+--------------+--------------+--------------------+--------------------+--------------+---------------+--------+--------------------+-----------------+----------------------+----+--------------------+-----+--------------+---------------+----------+--------------------+------+----------+-------------+------------+--------------------+----+-----+
|          CNPJ|        NOME_EMPRESA|            FANTASIA|   NATUREZA_JURIDICA| ATIVIDADE_PRINCIPAL|ATIVIDADES_SECUNDARIAS|    CNPJ_FORMATADO|  TIPO|      LOGRADOURO|NUMERO|COMPLEMENTO|          BAIRRO|     CEP|     MUNICIPIO| UF|    TELEFONE_1|    TELEFONE_2|               EMAIL|            ABERTURA|CAPITAL_SOCIAL|MOTIVO_SITUACAO|SITUACAO|       DATA_SITUACAO|SITUACAO_ESPECIAL|DATA_SITUACAO_ESPECIAL| EFR|          NOME

In [10]:
#------ Convertendo REF_DATE do df_cursor para timestamp

df_cursor = df_cursor.withColumn('REF_DATE',SF.from_unixtime(
            SF.unix_timestamp('DATA_REF','yyyy-MM')).cast('timestamp'))

#df_cursor.show(5)

In [11]:
#------ Convertendo REF_DATE do df para timestamp

df = df.withColumn('REF_DATE',SF.col('REF_DATE').cast('timestamp'))

In [12]:
df = df.withColumnRenamed('CNPJ','CNPJ_DF').withColumnRenamed('REF_DATE','REF_DATE_DF')

In [13]:
#------ Criando o df_base. O df_base é construído a partir do cursor. Todas as entradas do cursor são preservadas 
# e é feito um join com as entradas do df que correspondem à condição dupla: mesmo "CNPJ" e "Data de Referência" do df
# anterior à "Dara de Referência" do cursor  

df_base = df_cursor.join(df,
                 (df_cursor['CNPJ']==df['CNPJ_DF']) & 
                 (df_cursor['REF_DATE'] >= df['REF_DATE_DF']),
                 'left')

<h1>?) Classificação por CEP : Região, Subregião, Setor, Subsetor, Divisão de Subsetor e Logradouro <colour:red>(Falta decidir se o CEP deve entrar e com quantos números)</colour></h1>

In [16]:
#df_aux = df_base.select(['CNPJ', 'REF_DATE','CEP']).dropDuplicates()

In [17]:
'''
df_aux = df_aux.withColumn('REGIAO',SF.col('CEP').substr(1, 1)).\
         withColumn('SUBREGIAO',SF.col('CEP').substr(1, 2)).\
         withColumn('SETOR',SF.col('CEP').substr(1, 3)).\
         withColumn('SUBSETOR',SF.col('CEP').substr(1, 4)).\
         withColumn('DIV_SUBSETOR',SF.col('CEP').substr(1, 5)).\
         withColumn('LOGRADOURO',SF.col('CEP').substr(1, 9)).\
         drop('CEP')
'''

"\ndf_aux = df_aux.withColumn('REGIAO',SF.col('CEP').substr(1, 1)).         withColumn('SUBREGIAO',SF.col('CEP').substr(1, 2)).         withColumn('SETOR',SF.col('CEP').substr(1, 3)).         withColumn('SUBSETOR',SF.col('CEP').substr(1, 4)).         withColumn('DIV_SUBSETOR',SF.col('CEP').substr(1, 5)).         withColumn('LOGRADOURO',SF.col('CEP').substr(1, 9)).         drop('CEP')\n"

In [18]:
#df_aux.show()

In [19]:
#df_final = df_cursor.join(df_aux,['CNPJ','REF_DATE'],'left')
#df_final.show()

#df_final = df_final.join(df_aux, ['CNPJ','REF_DATE'], 'left')
#df_final.show()

In [20]:
#df_final.groupBy('REGIAO').count().toPandas()

<h1>1) Quantidade de Sócios</h1>

In [14]:
df_aux = df_base.select('CNPJ','REF_DATE','NOME_SOCIO').dropDuplicates().\
            groupBy('CNPJ','REF_DATE').agg(SF.count('NOME_SOCIO').alias('QTD_SOCIOS')).sort(SF.desc('QTD_SOCIOS'))
#df_aux.show(10)

In [15]:
df_final = df_cursor.join(df_aux, ['CNPJ','REF_DATE'], 'left')
#df_final.toPandas()

<h1> 2) Quantidade de Filiais do Grupo</h1>

In [23]:
'''df_aux = df_base.select('CNPJ','REF_DATE').dropDuplicates()
df_aux = df_aux.withColumn('GRUPO_EMP',SF.col('CNPJ').substr(1, 8)).\
         withColumn('FILIAL',SF.col('CNPJ').substr(9, 4)).\
         groupBy('CNPJ','REF_DATE','GRUPO_EMP').agg(SF.max('FILIAL').alias('QTD_FILIAIS')).drop('GRUPO_EMP').sort(SF.desc('QTD_FILIAIS'))



df_aux.show(10)
'''

"df_aux = df_base.select('CNPJ','REF_DATE').dropDuplicates()\ndf_aux = df_aux.withColumn('GRUPO_EMP',SF.col('CNPJ').substr(1, 8)).         withColumn('FILIAL',SF.col('CNPJ').substr(9, 4)).         groupBy('CNPJ','REF_DATE','GRUPO_EMP').agg(SF.max('FILIAL').alias('QTD_FILIAIS')).drop('GRUPO_EMP').sort(SF.desc('QTD_FILIAIS'))\n\n\n\ndf_aux.show(10)\n"

In [24]:
#df_final = df_final.join(df_aux, ['CNPJ','REF_DATE'], 'left')
#df_final.limit(10).toPandas()

In [25]:
#df_final.orderBy("QTD_FILIAIS", ascending=False).show(10)

<h1> 3) Natureza Jurídica </h1>

In [16]:
df_aux = df_base.select('CNPJ','REF_DATE','NATUREZA_JURIDICA').dropDuplicates()
#df_aux.limit(20).toPandas()

In [17]:
df_final = df_final.join(df_aux,['CNPJ','REF_DATE'], 'left' )
#df_final.limit(10).toPandas()

<h1>4) Idade da Empresa </h1>

In [18]:
#Aqui estamos incluindo a idade em segundos (como no final vamos precisar normalizar, não faz diferença)
df_aux = df_base.select('CNPJ','REF_DATE','ABERTURA').dropDuplicates()
df_aux = df_aux.withColumn('ABERTURA',SF.col('ABERTURA').cast('timestamp'))
df_aux = df_aux.withColumn('IDADE',SF.col('REF_DATE').cast("long")-SF.col('ABERTURA').cast("long")).drop('ABERTURA')

#df_aux.show(5)

In [19]:
df_final = df_final.join(df_aux,['CNPJ','REF_DATE'], 'left' )
#df_final.limit(10).toPandas()

<h1>5) Capital Social </h1>

In [30]:
#df.select('CAPITAL_SOCIAL').summary().show()

+-------+-------------------+
|summary|     CAPITAL_SOCIAL|
+-------+-------------------+
|  count|             214685|
|   mean| 1675295.2094820316|
| stddev|7.994737990014492E7|
|    min|                0.0|
|    25%|             5000.0|
|    50%|            20000.0|
|    75%|            60000.0|
|    max|          9999944.0|
+-------+-------------------+



In [31]:
#df.select('CAPITAL_SOCIAL','NATUREZA_JURIDICA').filter(SF.col('CAPITAL_SOCIAL')==0).show(20,False)

+--------------+-----------------------------+
|CAPITAL_SOCIAL|NATUREZA_JURIDICA            |
+--------------+-----------------------------+
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |SOCIEDADE EMPRESARIA LIMITADA|
|0.0           |EMPRESARIO (INDIVIDUAL)      |
|0.0           |EMPRESARIO (INDIVIDUAL)      |
|0.0         

In [20]:
df_aux = df_base.select('CNPJ','REF_DATE','CAPITAL_SOCIAL').dropDuplicates()
#df_aux.show(50)

In [21]:
df_final = df_final.join(df_aux,['CNPJ','REF_DATE'], 'left' )
#df_final.limit(10).toPandas()

<h1>6) Tipo (Matriz ou Filial) </h1>

In [22]:
df_aux = df_base.select('CNPJ','REF_DATE','TIPO').dropDuplicates()
df_final = df_final.join(df_aux,['CNPJ','REF_DATE'], 'left' )
#df_final.limit(10).toPandas()

In [35]:
df_final.show(100,False)

+--------------+-------------------+--------+-------+----------+------------------------------------------------------------------------+----------+--------------+------+
|CNPJ          |REF_DATE           |DATA_REF|CS_ALVO|QTD_SOCIOS|NATUREZA_JURIDICA                                                       |IDADE     |CAPITAL_SOCIAL|TIPO  |
+--------------+-------------------+--------+-------+----------+------------------------------------------------------------------------+----------+--------------+------+
|00004330000155|2016-10-01 00:00:00|2016-10 |0      |1         |EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA (DE NATUREZA EMPRESARIA)|706244400 |90000.0       |MATRIZ|
|00078090000133|2017-03-01 00:00:00|2017-03 |0      |2         |SOCIEDADE EMPRESARIA LIMITADA                                           |718081200 |0.0           |MATRIZ|
|00109982000154|2016-09-01 00:00:00|2016-09 |1      |2         |SOCIEDADE EMPRESARIA LIMITADA                                           |69881400

<h1>7) Situação </h1>

In [47]:
#df_aux = df_base.select('CNPJ','REF_DATE','SITUACAO').dropDuplicates()
#df_final = df_final.join(df_aux,['CNPJ','REF_DATE'], 'left' )

df_aux = df_base.select('CNPJ','REF_DATE','SITUACAO').dropDuplicates().\
            groupBy('CNPJ','REF_DATE','SITUACAO').agg(SF.max('REF_DATE').alias('MAX_REF_DATE'))

df_aux.show(10)

+--------------+-------------------+--------+-------------------+
|          CNPJ|           REF_DATE|SITUACAO|       MAX_REF_DATE|
+--------------+-------------------+--------+-------------------+
|04309109000166|2016-10-01 00:00:00|   ATIVA|2016-10-01 00:00:00|
|07115699000184|2016-09-01 00:00:00|   ATIVA|2016-09-01 00:00:00|
|82688128000154|2017-02-01 00:00:00|   ATIVA|2017-02-01 00:00:00|
|16571550000109|2017-01-01 00:00:00| BAIXADA|2017-01-01 00:00:00|
|15558469000118|2017-04-01 00:00:00|   ATIVA|2017-04-01 00:00:00|
|04781776000147|2017-03-01 00:00:00|   ATIVA|2017-03-01 00:00:00|
|07909991000179|2017-03-01 00:00:00|   ATIVA|2017-03-01 00:00:00|
|08365582000111|2016-09-01 00:00:00|   ATIVA|2016-09-01 00:00:00|
|22851094000145|2016-09-01 00:00:00|   ATIVA|2016-09-01 00:00:00|
|20599750000139|2017-03-01 00:00:00|   ATIVA|2017-03-01 00:00:00|
+--------------+-------------------+--------+-------------------+
only showing top 10 rows



In [28]:
df_aux.groupBy('CNPJ').count().orderBy("count", ascending=0).show(10)

+--------------+-----+
|          CNPJ|count|
+--------------+-----+
|07558813000140|    1|
|15217663000130|    1|
|11113907000183|    1|
|10710709000134|    1|
|10449557000168|    1|
|00305749000147|    1|
|33613845000101|    1|
|07446344000178|    1|
|02323877000149|    1|
|22469189000107|    1|
+--------------+-----+
only showing top 10 rows



In [48]:
df_aux = df_aux.drop("REF_DATE").dropDuplicates().withColumnRenamed("MAX_REF_DATE","REF_DATE")
df_aux.show(20)

+--------------+--------+-------------------+
|          CNPJ|SITUACAO|           REF_DATE|
+--------------+--------+-------------------+
|09137552000110|   ATIVA|2017-02-01 00:00:00|
|15538995000116|   ATIVA|2016-11-01 00:00:00|
|97385470000152|   ATIVA|2016-11-01 00:00:00|
|14419906000150|   ATIVA|2017-03-01 00:00:00|
|66521220000179|   ATIVA|2016-12-01 00:00:00|
|48157564000169|   ATIVA|2016-09-01 00:00:00|
|01928075000108|   ATIVA|2016-08-01 00:00:00|
|67558866000193|   ATIVA|2017-01-01 00:00:00|
|23960650000184| BAIXADA|2017-02-01 00:00:00|
|00382316000195|   ATIVA|2016-10-01 00:00:00|
|15370185000101|   ATIVA|2017-05-01 00:00:00|
|11506682000125|   ATIVA|2017-01-01 00:00:00|
|21035139000140|   ATIVA|2017-03-01 00:00:00|
|22917821000120|   ATIVA|2016-09-01 00:00:00|
|13710410000178|   ATIVA|2017-03-01 00:00:00|
|00630419000127|   ATIVA|2017-05-01 00:00:00|
|20591378000114|   ATIVA|2016-11-01 00:00:00|
|04697015000101|   ATIVA|2016-10-01 00:00:00|
|00276285000198|   ATIVA|2017-01-0

In [49]:
df_final = df_final.join(df_aux,['CNPJ','REF_DATE'], 'left' )

<h1>8) Atividade Principal </h1>

In [31]:
df_aux = df_base.select('CNPJ','REF_DATE','ATIVIDADE_PRINCIPAL').dropDuplicates()
df_final = df_final.join(df_aux,['CNPJ','REF_DATE'], 'left' )
#df_final.limit(20).toPandas()

In [50]:
df_teste = df_final

In [44]:
df_teste.show(5)

+--------------+-------------------+--------+-------+----------+--------------------+---------+--------------+------+--------+--------------------+
|          CNPJ|           REF_DATE|DATA_REF|CS_ALVO|QTD_SOCIOS|   NATUREZA_JURIDICA|    IDADE|CAPITAL_SOCIAL|  TIPO|SITUACAO| ATIVIDADE_PRINCIPAL|
+--------------+-------------------+--------+-------+----------+--------------------+---------+--------------+------+--------+--------------------+
|00004330000155|2016-10-01 00:00:00| 2016-10|      0|         1|EMPRESA INDIVIDUA...|706244400|       90000.0|MATRIZ|   ATIVA|RESTAURANTES E SI...|
|00078090000133|2017-03-01 00:00:00| 2017-03|      0|         2|SOCIEDADE EMPRESA...|718081200|           0.0|MATRIZ|   ATIVA|BARES E OUTROS ES...|
|00109982000154|2016-09-01 00:00:00| 2016-09|      1|         2|SOCIEDADE EMPRESA...|698814000|       10000.0|MATRIZ|   ATIVA|COMERCIO VAREJIST...|
|00281129000115|2017-06-01 00:00:00| 2017-06|      0|         3|SOCIEDADE EMPRESA...|712465200|       30000.0|MA

In [36]:
df_teste.groupBy("NATUREZA_JURIDICA").count().orderBy("count",ascending=0).show()

+--------------------+-----+
|   NATUREZA_JURIDICA|count|
+--------------------+-----+
|SOCIEDADE EMPRESA...|37550|
|EMPRESARIO (INDIV...|20956|
|EMPRESA INDIVIDUA...| 6585|
|SOCIEDADE ANONIMA...|   63|
|         COOPERATIVA|   47|
|SOCIEDADE SIMPLES...|   35|
|  ASSOCIACAO PRIVADA|   10|
|SERVICO SOCIAL AU...|    7|
|SOCIEDADE SIMPLES...|    6|
|   ENTIDADE SINDICAL|    5|
|SOCIEDADE EMPRESA...|    4|
|                null|    3|
|EMPRESA INDIVIDUA...|    3|
|    FUNDACAO PRIVADA|    2|
|ORGANIZACAO RELIG...|    1|
|SOCIEDADE ANONIMA...|    1|
+--------------------+-----+



In [51]:
df_teste.groupBy("SITUACAO").count().orderBy("count",ascending=0).show()

+--------+-----+
|SITUACAO|count|
+--------+-----+
|   ATIVA|60293|
| BAIXADA| 4425|
|  INAPTA|  475|
|SUSPENSA|   86|
|    null|    3|
+--------+-----+



<h1>9) Unidade Federativa </h1>

In [39]:
#df.groupby('UF').count().orderBy('count').show(30)

In [40]:
#df_teste = df_final

In [41]:
#df_teste.show(10)

In [52]:
df_aux = df_base.select('CNPJ','REF_DATE','UF').dropDuplicates()
df_final = df_final.join(df_aux,['CNPJ','REF_DATE'], 'left' )
df_final.show(10)

+--------------+-------------------+--------+-------+----------+--------------------+----------+--------------+------+--------------------+--------+---+
|          CNPJ|           REF_DATE|DATA_REF|CS_ALVO|QTD_SOCIOS|   NATUREZA_JURIDICA|     IDADE|CAPITAL_SOCIAL|  TIPO| ATIVIDADE_PRINCIPAL|SITUACAO| UF|
+--------------+-------------------+--------+-------+----------+--------------------+----------+--------------+------+--------------------+--------+---+
|00004330000155|2016-10-01 00:00:00| 2016-10|      0|         1|EMPRESA INDIVIDUA...| 706244400|       90000.0|MATRIZ|RESTAURANTES E SI...|   ATIVA| SP|
|00078090000133|2017-03-01 00:00:00| 2017-03|      0|         2|SOCIEDADE EMPRESA...| 718081200|           0.0|MATRIZ|BARES E OUTROS ES...|   ATIVA| RS|
|00109982000154|2016-09-01 00:00:00| 2016-09|      1|         2|SOCIEDADE EMPRESA...| 698814000|       10000.0|MATRIZ|COMERCIO VAREJIST...|   ATIVA| SP|
|00281129000115|2017-06-01 00:00:00| 2017-06|      0|         3|SOCIEDADE EMPRESA.

In [54]:
df_teste = df_final
df_teste.groupBy("UF").count().orderBy("count",ascending=0).show()

+----+-----+
|  UF|count|
+----+-----+
|  SP|24533|
|  RJ| 6492|
|  MG| 6109|
|  PR| 4519|
|  RS| 4286|
|null| 3716|
|  BA| 2646|
|  SC| 2476|
|  PE| 1667|
|  GO| 1294|
|  DF| 1278|
|  ES| 1247|
|  CE| 1107|
|  MS|  502|
|  MT|  492|
|  PA|  434|
|  PB|  414|
|  AM|  375|
|  MA|  328|
|  RN|  262|
+----+-----+
only showing top 20 rows



In [43]:
'''
Próximos passos:
1 - Analisar cada variável, tratar os nulos, os outliers e decidir se a variável continua no modelo;
2 - Transformar as variáveis categóricas e numéricas (Petronio indicou usar catBoostEncoder)
3 - Treinar o modelo (vamos usar Spark ou pandas?)
4 - Testar o modelo
'''

'\nPróximos passos:\n1 - Analisar cada variável, tratar os nulos, os outliers e decidir se a variável continua no modelo;\n2 - Transformar as variáveis categóricas e numéricas (Petronio indicou usar catBoostEncoder)\n3 - Treinar o modelo (vamos usar Spark ou pandas?)\n4 - Testar o modelo\n'